# Objetivos

* Seleccionar el mejor modelo desarrollado por Juan en el notebook llamado "to-expose"
* Probar tecnicas adicionales esperando encontrar mejoras en los resultados

## Sub-objetivos

* Limpiar metodos para que sea más simple llevar a producción

# Seleccionar el mejor modelo desarrollado por Juan

Ambos modelos presentan resultados similares en las pruebas realizadas por Juan. Además, puedo observar que no se han resuelto algunas problemáticas importantes, tales como:<br>
* ¿Los modelos tienen mucha variabilidad o sesgo?<br>
* ¿Cuál es la métrica más importante?<br>
* ¿Cómo solucionar el problema de desbalance en los datos?<br>
* ¿Que resultados obtendriamos si probamos el origen y destino de los vuelos como características? En los gráficos presentados parecen tener relevancia.<br>
* ¿Existe algún dato que se pueda sumar a partir de las ciudades de origen y destino?
<br>
Por lo tanto, considero necesario realizar los siguientes pasos y elecciones:<br>
<br>
* XGBoost es una buena elección, ya que se puede utilizar para selección binaria y es ideal para datasets de alta dimensionalidad, como es este caso.<br>
* La métrica más importante es probablemente el F1-score, dado el desequilibrio en la variable de target, no queremos que el modelo tenga un alto número de falsos positivos ni de falsos negativos, <br>por lo que F1-score es una buena opción ya que considera tanto la precisión como el recall.<br>
* Para solucionar el problema del desbalance, podemos probar con los hiperparámetros scale_pos_weight y class_weight para asignar un peso a cada clase.<br>
* Agregar más características. En caso de ser necesario, podemos reducirlas utilizando PCA y/o seleccionando las características más importantes.<br>
* No se buscaran features de datos externos dado el tiempo disponible para el challenge



# Setup

In [1]:
#librerias para el manejo de los paths
import os
from pathlib import Path

#librerías de tratamiento de datos
import pandas as pd
import numpy as np

#Librerias de visualización
import matplotlib.pyplot as plt
import seaborn as sns

#Librerias propias de python
from datetime import datetime as dt
from datetime import date

#Modelos y metricas
import xgboost as xgb
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.model_selection import GridSearchCV

#Preprocesamiento
from sklearn.preprocessing import OneHotEncoder
import category_encoders as ce
from sklearn.preprocessing import LabelEncoder
from fast_ml.model_development import train_valid_test_split
import random

In [2]:
#Se obtiene el path a data
notebooks_path = os.getcwd()
main_path = Path(notebooks_path).parent
data_path = main_path / "data"

# Read raw file

In [3]:
raw_data_path = data_path / "raw/dataset_SCL.csv"
raw_data = pd.read_csv(raw_data_path)
raw_data.shape

(68206, 18)

In [4]:
raw_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 68206 entries, 0 to 68205
Data columns (total 18 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   Fecha-I    68206 non-null  object
 1   Vlo-I      68206 non-null  object
 2   Ori-I      68206 non-null  object
 3   Des-I      68206 non-null  object
 4   Emp-I      68206 non-null  object
 5   Fecha-O    68206 non-null  object
 6   Vlo-O      68205 non-null  object
 7   Ori-O      68206 non-null  object
 8   Des-O      68206 non-null  object
 9   Emp-O      68206 non-null  object
 10  DIA        68206 non-null  int64 
 11  MES        68206 non-null  int64 
 12  AÑO        68206 non-null  int64 
 13  DIANOM     68206 non-null  object
 14  TIPOVUELO  68206 non-null  object
 15  OPERA      68206 non-null  object
 16  SIGLAORI   68206 non-null  object
 17  SIGLADES   68206 non-null  object
dtypes: int64(3), object(15)
memory usage: 9.4+ MB


In [5]:
# Se copia el dataframe para probar más facilmente los métodos originales
df = raw_data.copy(deep=True)

# Métodos

Se contrastan los métodos originales con los nuevos tanto en clean code como en tiempos de operación. <br>
Queda tener en cuenta que los tiempos varian según el procesador pero se busca dar un resultado aproximado de la diferencia de ambos metodos.

## Calculo si el vuelo llego atrazado o no

In [6]:
# Metodo original
def dif_min(data):
    fecha_o = dt.strptime(data['Fecha-O'], '%Y-%m-%d %H:%M:%S')
    fecha_i = dt.strptime(data['Fecha-I'], '%Y-%m-%d %H:%M:%S')
    dif_min = ((fecha_o - fecha_i).total_seconds())/60
    return dif_min

df['dif_min'] = df.apply(dif_min, axis = 1)
df['atraso_15'] = np.where(df['dif_min'] > 15, 1, 0)

In [8]:
# Nuevo metodo
raw_data = raw_data.astype({'Fecha-O': "datetime64", 'Fecha-I': "datetime64"})

def add_delay_status_column(raw_data: pd.DataFrame) -> pd.DataFrame:
    time_difference = raw_data['Fecha-O'] - raw_data['Fecha-I']
    minutes_difference = time_difference.dt.seconds.div(60).astype(int)
    raw_data['delay_status'] = minutes_difference > 15
    raw_data.loc[raw_data['Fecha-O'] < raw_data['Fecha-I'], 'delay_status'] = False
    return raw_data

test = add_delay_status_column(raw_data=raw_data).astype({'delay_status': int})

El método nuevo resulta ser aprox 766% más rápido

## Encontrar si es temporada alta

In [12]:
# Método original
def temporada_alta(fecha):
    fecha_año = int(fecha.split('-')[0])
    fecha = dt.strptime(fecha, '%Y-%m-%d %H:%M:%S')
    range1_min = dt.strptime('15-Dec', '%d-%b').replace(year = fecha_año)
    range1_max = dt.strptime('31-Dec', '%d-%b').replace(year = fecha_año)
    range2_min = dt.strptime('1-Jan', '%d-%b').replace(year = fecha_año)
    range2_max = dt.strptime('3-Mar', '%d-%b').replace(year = fecha_año)
    range3_min = dt.strptime('15-Jul', '%d-%b').replace(year = fecha_año)
    range3_max = dt.strptime('31-Jul', '%d-%b').replace(year = fecha_año)
    range4_min = dt.strptime('11-Sep', '%d-%b').replace(year = fecha_año)
    range4_max = dt.strptime('30-Sep', '%d-%b').replace(year = fecha_año)
    
    if ((fecha >= range1_min and fecha <= range1_max) or 
        (fecha >= range2_min and fecha <= range2_max) or 
        (fecha >= range3_min and fecha <= range3_max) or
        (fecha >= range4_min and fecha <= range4_max)):
        return 1
    else:
        return 0

df['temporada_alta'] = df['Fecha-I'].apply(temporada_alta)

In [13]:
#Método nuevo
raw_data = raw_data.astype({'Fecha-I': "datetime64"})

def add_season_status_column(raw_data: pd.DataFrame) -> pd.DataFrame:
    # High Season cases:
    # Dec 15 to Dec 31
    # Jan 1 to Mar 3
    # Jul 15 to Jul 31
    # Sep 11 to Sep 30 
    high_season_ranges = [
        ((12, 15), (12, 31)),
        ((1, 1), (3, 3)),
        ((7, 15), (7, 31)),
        ((9, 11), (9, 30))
    ]
    
    raw_data['is_high_season'] = raw_data['Fecha-I'].apply(
        lambda date: any(
            start <= (date.month, date.day) <= end
            for start, end in high_season_ranges
        )
    )
    
    return raw_data

test = add_season_status_column(raw_data=raw_data).astype({'is_high_season': int})

El método nuevo resulta ser aprox 1225% más rápido 

In [14]:
# Método original
def get_periodo_dia(fecha):
    fecha_time = dt.strptime(fecha, '%Y-%m-%d %H:%M:%S').time()
    mañana_min = dt.strptime("05:00", '%H:%M').time()
    mañana_max = dt.strptime("11:59", '%H:%M').time()
    tarde_min = dt.strptime("12:00", '%H:%M').time()
    tarde_max = dt.strptime("18:59", '%H:%M').time()
    noche_min1 = dt.strptime("19:00", '%H:%M').time()
    noche_max1 = dt.strptime("23:59", '%H:%M').time()
    noche_min2 = dt.strptime("00:00", '%H:%M').time()
    noche_max2 = dt.strptime("4:59", '%H:%M').time()
    
    if(fecha_time > mañana_min and fecha_time < mañana_max):
        return 'mañana'
    elif(fecha_time > tarde_min and fecha_time < tarde_max):
        return 'tarde'
    elif((fecha_time > noche_min1 and fecha_time < noche_max1) or
         (fecha_time > noche_min2 and fecha_time < noche_max2)):
        return 'noche'

df['periodo_dia'] = df['Fecha-I'].apply(get_periodo_dia)

In [15]:
#Método nuevo
raw_data = raw_data.astype({'Fecha-I': "datetime64"})

def add_day_period_column(raw_data: pd.DataFrame) -> pd.DataFrame:
    day_periods = {
        'morning': (5, 12),
        'afternoon': (12, 19),
        'evening_one': (19, 24),
        'evening_two': (0, 5)
    }
    
    raw_data['day_period'] = raw_data['Fecha-I'].apply(
        lambda datetime: next((period for period, hours in day_periods.items() if hours[0] <= datetime.hour < hours[1]), None)
    )

    raw_data.loc[raw_data['day_period'].isin(['evening_one', 'evening_two']), 'day_period'] = 'evening'
    
    return raw_data

test = add_day_period_column(raw_data=raw_data)

El método nuevo resulta ser aprox 1566% más rápido 

## Preproceso

Bajo lo visto en "to-expose" se descartan las columnas 'Ori-I', 'Ori-O','Vlo-I', 'Vlo-O', 'OPERA', 'SIGLAORI', 'SIGLADES' ya que alguna de estas se repiten en forma de codigo en otras columnas<br>y algunas no parecen tener importancia. (Estas ultimas se podrian revisar en el caso de que este fuera un analysis más extenso)

In [16]:
def preprocess_data(raw_data: pd.DataFrame) -> pd.DataFrame:
    preprocessed_data = (
        raw_data
        .drop(columns=['Ori-I', 'Ori-O', 'Vlo-I', 'Vlo-O', 'OPERA', 'SIGLAORI', 'SIGLADES'], errors='ignore')
        .dropna()
        .astype({
            'Fecha-I': "datetime64",
            'Fecha-O': "datetime64",
        })
    )
    return preprocessed_data

## Process

In [17]:
def process_data(preprocessed_data: pd.DataFrame) -> pd.DataFrame:
    processed_data = (
        preprocessed_data
        .pipe(add_day_period_column)
        .pipe(add_season_status_column)
        .pipe(add_delay_status_column)
        .drop(columns=['Fecha-I', 'Fecha-O'], errors='ignore')
    )
    return processed_data

## Build Encoded features

In [18]:
preprocessed_data = preprocess_data(raw_data=raw_data)
processed_data = process_data(preprocessed_data=preprocessed_data)

In [19]:
X_train, y_train, X_valid, y_valid, X_test, y_test = train_valid_test_split(processed_data, target='delay_status', train_size=0.8, valid_size=0.1, test_size=0.1)

In [20]:
def transform_to_numpy(X: pd.DataFrame, columns: list) -> np.array:
    return X.loc[:, columns].to_numpy()

def create_one_hot_encoder(X_train: pd.DataFrame, columns: list) -> OneHotEncoder:
    one_hot_encoder = OneHotEncoder(handle_unknown='ignore')
    one_hot_encoder.fit(X_train[columns])
    return one_hot_encoder

def create_label_encoders(columns_with_values: dict) -> dict:
    label_encoders = {}
    for column in columns_with_values.keys():
        le = LabelEncoder()
        le.fit(columns_with_values[column])
        label_encoders[column] = le
    return label_encoders

def encode_one_hot_columns(X: pd.DataFrame, one_hot_encoder: OneHotEncoder) -> np.array:
    columns = one_hot_encoder.feature_names_in_.tolist()
    return one_hot_encoder.transform(X[columns]).toarray()

def encode_label_columns(X: pd.DataFrame, label_encoders: dict) -> np.array:
    label_encodes = np.array([])
    for column in label_encoders.keys():
        results = label_encoders[column].transform(X[column])
        results = results.reshape(results.shape[0], 1)
        if label_encodes.size == 0: 
            label_encodes = results
            continue
        label_encodes = np.concatenate((label_encodes, results), axis=1)
    return label_encodes

def encode_dataset(X: pd.DataFrame, one_hot_encoder: OneHotEncoder, label_encoders: dict, columns_to_numpy: list) -> np.array:
    not_encoded_variables = transform_to_numpy(X=X, columns=columns_to_numpy)
    label_encodes = encode_label_columns(X=X, label_encoders=label_encoders)
    sparse_encodes = encode_one_hot_columns(X=X, one_hot_encoder=one_hot_encoder)
    return np.concatenate([not_encoded_variables, label_encodes, sparse_encodes], axis=1)

In [21]:
one_hot_columns = ['Des-I', 'Emp-I', 'Des-O', 'Emp-O', 'TIPOVUELO', 'is_high_season']
one_hot_encoder = create_one_hot_encoder(X_train=X_train, columns=one_hot_columns)

label_columns_with_values = {
    'DIANOM': ["Lunes", "Martes", "Miercoles", "Jueves", "Viernes", "Sabado", "Domingo"], 
    'day_period': ["morning", "afternoon", "evening"]
}
label_encoders = create_label_encoders(columns_with_values=label_columns_with_values)

columns_to_numpy = ['DIA', 'MES', 'AÑO']

X_train_encoded = encode_dataset(X=X_train, one_hot_encoder=one_hot_encoder, label_encoders=label_encoders, columns_to_numpy=columns_to_numpy)
X_valid_encoded = encode_dataset(X=X_valid, one_hot_encoder=one_hot_encoder, label_encoders=label_encoders, columns_to_numpy=columns_to_numpy)
X_test_encoded = encode_dataset(X=X_test, one_hot_encoder=one_hot_encoder, label_encoders=label_encoders, columns_to_numpy=columns_to_numpy)


# Models

El modelo en to-expose que alcanzo el mayor F1-score fue con XGBoost y alcanzo el valor de 0.30. Ahora procederemos a crear otro modelo base para también utilizarlo en la comparación

## Modelo random

In [22]:
class RandomClassifier:
    def fit(self, X, y):
        self.classes = y.unique()

    def predict(self, X):
        return [random.choice(self.classes) for _ in range(len(X))]

In [23]:
model = RandomClassifier()
model.fit(X_train_encoded, y_train)

In [24]:
y_pred = model.predict(X_train_encoded)
print(confusion_matrix(y_train, y_pred))
print(classification_report(y_train, y_pred))

[[22411 22110]
 [ 5040  5003]]
              precision    recall  f1-score   support

       False       0.82      0.50      0.62     44521
        True       0.18      0.50      0.27     10043

    accuracy                           0.50     54564
   macro avg       0.50      0.50      0.45     54564
weighted avg       0.70      0.50      0.56     54564



In [25]:
y_pred = model.predict(X_valid_encoded)
print(confusion_matrix(y_valid, y_pred))
print(classification_report(y_valid, y_pred))

[[2806 2772]
 [ 642  601]]
              precision    recall  f1-score   support

       False       0.81      0.50      0.62      5578
        True       0.18      0.48      0.26      1243

    accuracy                           0.50      6821
   macro avg       0.50      0.49      0.44      6821
weighted avg       0.70      0.50      0.56      6821



Con el modelo random podemos ver que alcanzamos un F1-score de 0.26 en el los datos de validación lo cual ya no es mejor. <br>
Por lo que el modelo base sera el XGBoost de to-expose que tiene un F1-score de 0.30.

A continuación, realizamos un grid search para buscar los parametros iniciales del modelo XGBoost.

## Modelo XGBoost con one hot encoder

In [26]:
#To use cross validation
X_train_valid_encoded = np.concatenate((X_train_encoded, X_valid_encoded), axis=0)
y_train_valid = np.concatenate((y_train, y_valid), axis=0)

In [90]:
parameters_grid = {'max_depth': [4, 6, 20], 
                   'learning_rate': [0.1, 0.3],
                   'n_estimators': [50, 100], 
                   'gamma': [0, 20], 
                   'lambda': [0.1, 1],
                   'scale_pos_weight': [5, 10],
                   'tree_method': ['hist']
                   }

model = xgb.XGBClassifier()

xgb_gscv = GridSearchCV(model, parameters_grid, cv=2, scoring="f1", return_train_score=True, verbose=3, n_jobs=1)
xgb_gscv.fit(X_train_valid_encoded, y_train_valid)

Fitting 2 folds for each of 96 candidates, totalling 192 fits
[CV 1/2] END gamma=0, lambda=0.1, learning_rate=0.1, max_depth=4, n_estimators=50, scale_pos_weight=5, tree_method=hist;, score=(train=0.404, test=0.393) total time=   0.1s
[CV 2/2] END gamma=0, lambda=0.1, learning_rate=0.1, max_depth=4, n_estimators=50, scale_pos_weight=5, tree_method=hist;, score=(train=0.406, test=0.392) total time=   0.1s
[CV 1/2] END gamma=0, lambda=0.1, learning_rate=0.1, max_depth=4, n_estimators=50, scale_pos_weight=10, tree_method=hist;, score=(train=0.343, test=0.338) total time=   0.1s
[CV 2/2] END gamma=0, lambda=0.1, learning_rate=0.1, max_depth=4, n_estimators=50, scale_pos_weight=10, tree_method=hist;, score=(train=0.349, test=0.344) total time=   0.1s
[CV 1/2] END gamma=0, lambda=0.1, learning_rate=0.1, max_depth=4, n_estimators=100, scale_pos_weight=5, tree_method=hist;, score=(train=0.421, test=0.402) total time=   0.2s
[CV 2/2] END gamma=0, lambda=0.1, learning_rate=0.1, max_depth=4, n_es

GridSearchCV(cv=2,
             estimator=XGBClassifier(base_score=None, booster=None,
                                     callbacks=None, colsample_bylevel=None,
                                     colsample_bynode=None,
                                     colsample_bytree=None,
                                     early_stopping_rounds=None,
                                     enable_categorical=False, eval_metric=None,
                                     feature_types=None, gamma=None,
                                     gpu_id=None, grow_policy=None,
                                     importance_type=None,
                                     interaction_constraints=None,
                                     learning_rate=None,...
                                     max_leaves=None, min_child_weight=None,
                                     missing=nan, monotone_constraints=None,
                                     n_estimators=100, n_jobs=None,
                                     num_parallel_tree=None, predictor=None,
                                     random_state=None, ...),
             n_jobs=1,
             param_grid={'gamma': [0, 20], 'lambda': [0.1, 1],
                         'learning_rate': [0.1, 0.3], 'max_depth': [4, 6, 20],
                         'n_estimators': [50, 100], 'scale_pos_weight': [5, 10],
                         'tree_method': ['hist']},
             return_train_score=True, scoring='f1', verbose=3)

In [91]:
cv_results = xgb_gscv.cv_results_
cv_results = pd.concat([pd.DataFrame(cv_results["params"]),
                        pd.DataFrame(cv_results["mean_test_score"], columns=["test_f1"]),
                        pd.DataFrame(cv_results["mean_train_score"], columns=["train_f1"])],
                       axis=1)
print(cv_results.sort_values(by="test_f1", ascending=False))

    gamma  lambda  learning_rate  max_depth  n_estimators  scale_pos_weight  \
6       0     0.1            0.1          6           100                 5   
30      0     1.0            0.1          6           100                 5   
18      0     0.1            0.3          6           100                 5   
42      0     1.0            0.3          6           100                 5   
40      0     1.0            0.3          6            50                 5   
..    ...     ...            ...        ...           ...               ...   
25      0     1.0            0.1          4            50                10   
49     20     0.1            0.1          4            50                10   
51     20     0.1            0.1          4           100                10   
73     20     1.0            0.1          4            50                10   
75     20     1.0            0.1          4           100                10   

   tree_method   test_f1  train_f1  
6         hist

In [92]:
params = {        
    'max_depth': 20, 
    'learning_rate': 0.3,
    'n_estimators': 100, 
    'gamma': 20, 
    'lambda': 0.1,
    'scale_pos_weight': 5
}
model = xgb.XGBClassifier(**params)
model.fit(X_train_encoded, y_train)

print("Training results")
y_pred = model.predict(X_train_encoded)
print(confusion_matrix(y_train, y_pred))
print(classification_report(y_train, y_pred))

print("Validation results")
y_pred = model.predict(X_valid_encoded)
print(confusion_matrix(y_valid, y_pred))
print(classification_report(y_valid, y_pred))

Training results
[[32667 11854]
 [ 1612  8431]]
              precision    recall  f1-score   support

       False       0.95      0.73      0.83     44521
        True       0.42      0.84      0.56     10043

    accuracy                           0.75     54564
   macro avg       0.68      0.79      0.69     54564
weighted avg       0.85      0.75      0.78     54564

Validation results
[[3851 1727]
 [ 442  801]]
              precision    recall  f1-score   support

       False       0.90      0.69      0.78      5578
        True       0.32      0.64      0.42      1243

    accuracy                           0.68      6821
   macro avg       0.61      0.67      0.60      6821
weighted avg       0.79      0.68      0.72      6821



El modelo mejora bastante en su F1-score de validación pero podemos ver que hay over-fitting.<br> 
Podriamos mover algunos hiper parametros para arreglar el over fitting pero primero probaremos XGBoost pero con category encoders en vez de one hot encoders

## Modelo XGBoost con category encoders

In [83]:
def create_category_encoder(X_train: pd.DataFrame, y_train: pd.DataFrame, columns: list) -> ce.GLMMEncoder:
    category_encoder = ce.GLMMEncoder(binomial_target = True)
    category_encoder.fit(X_train[columns], y_train)
    return category_encoder

def encode_category_columns(X: pd.DataFrame, category_encoder: ce.GLMMEncoder) -> np.array:
    columns = category_encoder.feature_names_out_
    return category_encoder.transform(X[columns])

def encode_dataset(X: pd.DataFrame, category_encoder: ce.GLMMEncoder, label_encoders: dict, columns_to_numpy: list) -> np.array:
    not_encoded_variables = transform_to_numpy(X=X, columns=columns_to_numpy)
    label_encodes = encode_label_columns(X=X, label_encoders=label_encoders)
    category_encodes = encode_category_columns(X=X, category_encoder=category_encoder)
    return np.concatenate([not_encoded_variables, label_encodes, category_encodes], axis=1)

In [85]:
category_encoders_columns = ['Des-I', 'Emp-I', 'Des-O', 'Emp-O', 'TIPOVUELO', 'is_high_season']
category_encoder = create_category_encoder(X_train=X_train, y_train=y_train, columns=category_encoders_columns)

label_columns_with_values = {
    'DIANOM': ["Lunes", "Martes", "Miercoles", "Jueves", "Viernes", "Sabado", "Domingo"], 
    'day_period': ["morning", "afternoon", "evening"]
}
label_encoders = create_label_encoders(columns_with_values=label_columns_with_values)

columns_to_numpy = ['DIA', 'MES', 'AÑO']

X_train_encoded = encode_dataset(X=X_train, category_encoder=category_encoder, label_encoders=label_encoders, columns_to_numpy=columns_to_numpy)
X_valid_encoded = encode_dataset(X=X_valid, category_encoder=category_encoder, label_encoders=label_encoders, columns_to_numpy=columns_to_numpy)
X_test_encoded = encode_dataset(X=X_test, category_encoder=category_encoder, label_encoders=label_encoders, columns_to_numpy=columns_to_numpy)

In [86]:
#To use cross validation
X_train_valid_encoded = np.concatenate((X_train_encoded, X_valid_encoded), axis=0)
y_train_valid = np.concatenate((y_train, y_valid), axis=0)

In [87]:
parameters_grid = {'max_depth': [4, 6, 20], 
                   'learning_rate': [0.1, 0.3],
                   'n_estimators': [50, 100], 
                   'gamma': [0, 20], 
                   'lambda': [0.1, 1],
                   'scale_pos_weight': [5, 10],
                   'tree_method': ['hist']}

model = xgb.XGBClassifier()

xgb_gscv = GridSearchCV(model, parameters_grid, cv=2, scoring="f1", return_train_score=True, verbose=3, n_jobs=1)
xgb_gscv.fit(X_train_valid_encoded, y_train_valid)

Fitting 2 folds for each of 96 candidates, totalling 192 fits
[CV 1/2] END gamma=0, lambda=0.1, learning_rate=0.1, max_depth=4, n_estimators=50, scale_pos_weight=5, tree_method=hist;, score=(train=0.404, test=0.393) total time=   0.2s
[CV 2/2] END gamma=0, lambda=0.1, learning_rate=0.1, max_depth=4, n_estimators=50, scale_pos_weight=5, tree_method=hist;, score=(train=0.406, test=0.392) total time=   0.1s
[CV 1/2] END gamma=0, lambda=0.1, learning_rate=0.1, max_depth=4, n_estimators=50, scale_pos_weight=10, tree_method=hist;, score=(train=0.343, test=0.338) total time=   0.1s
[CV 2/2] END gamma=0, lambda=0.1, learning_rate=0.1, max_depth=4, n_estimators=50, scale_pos_weight=10, tree_method=hist;, score=(train=0.349, test=0.344) total time=   0.0s
[CV 1/2] END gamma=0, lambda=0.1, learning_rate=0.1, max_depth=4, n_estimators=100, scale_pos_weight=5, tree_method=hist;, score=(train=0.421, test=0.402) total time=   0.1s
[CV 2/2] END gamma=0, lambda=0.1, learning_rate=0.1, max_depth=4, n_es

GridSearchCV(cv=2,
             estimator=XGBClassifier(base_score=None, booster=None,
                                     callbacks=None, colsample_bylevel=None,
                                     colsample_bynode=None,
                                     colsample_bytree=None,
                                     early_stopping_rounds=None,
                                     enable_categorical=False, eval_metric=None,
                                     feature_types=None, gamma=None,
                                     gpu_id=None, grow_policy=None,
                                     importance_type=None,
                                     interaction_constraints=None,
                                     learning_rate=None,...
                                     max_leaves=None, min_child_weight=None,
                                     missing=nan, monotone_constraints=None,
                                     n_estimators=100, n_jobs=None,
                                     num_parallel_tree=None, predictor=None,
                                     random_state=None, ...),
             n_jobs=1,
             param_grid={'gamma': [0, 20], 'lambda': [0.1, 1],
                         'learning_rate': [0.1, 0.3], 'max_depth': [4, 6, 20],
                         'n_estimators': [50, 100], 'scale_pos_weight': [5, 10],
                         'tree_method': ['hist']},
             return_train_score=True, scoring='f1', verbose=3)

In [88]:
cv_results = xgb_gscv.cv_results_
cv_results = pd.concat([pd.DataFrame(cv_results["params"]),
                        pd.DataFrame(cv_results["mean_test_score"], columns=["test_f1"]),
                        pd.DataFrame(cv_results["mean_train_score"], columns=["train_f1"])],
                       axis=1)
print(cv_results.sort_values(by="test_f1", ascending=False))

    gamma  lambda  learning_rate  max_depth  n_estimators  scale_pos_weight  \
6       0     0.1            0.1          6           100                 5   
30      0     1.0            0.1          6           100                 5   
18      0     0.1            0.3          6           100                 5   
42      0     1.0            0.3          6           100                 5   
40      0     1.0            0.3          6            50                 5   
..    ...     ...            ...        ...           ...               ...   
25      0     1.0            0.1          4            50                10   
49     20     0.1            0.1          4            50                10   
51     20     0.1            0.1          4           100                10   
73     20     1.0            0.1          4            50                10   
75     20     1.0            0.1          4           100                10   

   tree_method   test_f1  train_f1  
6         hist

In [89]:
params = {        
    'max_depth': 6, 
    'learning_rate': 0.1,
    'n_estimators': 100, 
    'gamma': 0, 
    'lambda': 0.1,
    'scale_pos_weight': 5
}
model = xgb.XGBClassifier(**params)
model.fit(X_train_encoded, y_train)

print("Training results")
y_pred = model.predict(X_train_encoded)
print(confusion_matrix(y_train, y_pred))
print(classification_report(y_train, y_pred))

print("Validation results")
y_pred = model.predict(X_valid_encoded)
print(confusion_matrix(y_valid, y_pred))
print(classification_report(y_valid, y_pred))

Training results
[[28695 15826]
 [ 2409  7634]]
              precision    recall  f1-score   support

       False       0.92      0.64      0.76     44521
        True       0.33      0.76      0.46     10043

    accuracy                           0.67     54564
   macro avg       0.62      0.70      0.61     54564
weighted avg       0.81      0.67      0.70     54564

Validation results
[[3528 2050]
 [ 383  860]]
              precision    recall  f1-score   support

       False       0.90      0.63      0.74      5578
        True       0.30      0.69      0.41      1243

    accuracy                           0.64      6821
   macro avg       0.60      0.66      0.58      6821
weighted avg       0.79      0.64      0.68      6821



Podemos ver que aqui encontramos un resultado ligeramente peor, pero el over fitting se reduce bastante por lo que utilizaremos los category encoders.<br>
Ahora buscaremos solucionar el over fitting

In [96]:
parameters_grid = {'max_depth': [6], 
                   'learning_rate': [0.1],
                   'n_estimators': [100], 
                   'gamma': [0], 
                   'lambda': [0.1],
                   'scale_pos_weight': [5],
                   'subsample': [1, 0.9, 0.8, 0.7],
                   'max_leaves': [0, 8, 9, 10, 20],
                   'min_child_weight': [1, 0.9, 0.8, 0.7],
                   'tree_method': ['hist']
                   }

model = xgb.XGBClassifier()

xgb_gscv = GridSearchCV(model, parameters_grid, cv=2, scoring="f1", return_train_score=True, verbose=3, n_jobs=1)
xgb_gscv.fit(X_train_valid_encoded, y_train_valid)

Fitting 2 folds for each of 80 candidates, totalling 160 fits
[CV 1/2] END gamma=0, lambda=0.1, learning_rate=0.1, max_depth=6, max_leaves=0, min_child_weight=1, n_estimators=100, scale_pos_weight=5, subsample=1, tree_method=hist;, score=(train=0.476, test=0.415) total time=   0.3s
[CV 2/2] END gamma=0, lambda=0.1, learning_rate=0.1, max_depth=6, max_leaves=0, min_child_weight=1, n_estimators=100, scale_pos_weight=5, subsample=1, tree_method=hist;, score=(train=0.474, test=0.413) total time=   0.2s
[CV 1/2] END gamma=0, lambda=0.1, learning_rate=0.1, max_depth=6, max_leaves=0, min_child_weight=1, n_estimators=100, scale_pos_weight=5, subsample=0.9, tree_method=hist;, score=(train=0.483, test=0.415) total time=   0.3s
[CV 2/2] END gamma=0, lambda=0.1, learning_rate=0.1, max_depth=6, max_leaves=0, min_child_weight=1, n_estimators=100, scale_pos_weight=5, subsample=0.9, tree_method=hist;, score=(train=0.483, test=0.414) total time=   0.6s
[CV 1/2] END gamma=0, lambda=0.1, learning_rate=0.

GridSearchCV(cv=2,
             estimator=XGBClassifier(base_score=None, booster=None,
                                     callbacks=None, colsample_bylevel=None,
                                     colsample_bynode=None,
                                     colsample_bytree=None,
                                     early_stopping_rounds=None,
                                     enable_categorical=False, eval_metric=None,
                                     feature_types=None, gamma=None,
                                     gpu_id=None, grow_policy=None,
                                     importance_type=None,
                                     interaction_constraints=None,
                                     learning_rate=None,...
                                     num_parallel_tree=None, predictor=None,
                                     random_state=None, ...),
             n_jobs=1,
             param_grid={'gamma': [0], 'lambda': [0.1], 'learning_rate': [0.1],
                         'max_depth': [6], 'max_leaves': [0, 8, 9, 10, 20],
                         'min_child_weight': [1, 0.9, 0.8, 0.7],
                         'n_estimators': [100], 'scale_pos_weight': [5],
                         'subsample': [1, 0.9, 0.8, 0.7],
                         'tree_method': ['hist']},
             return_train_score=True, scoring='f1', verbose=3)

In [107]:
cv_results = xgb_gscv.cv_results_
cv_results = pd.concat([pd.DataFrame(cv_results["params"]),
                        pd.DataFrame(cv_results["mean_test_score"], columns=["test_f1"]),
                        pd.DataFrame(cv_results["mean_train_score"], columns=["train_f1"])],
                       axis=1)
print(cv_results.sort_values(by="test_f1", ascending=False).iloc[16])

gamma                      0
lambda                   0.1
learning_rate            0.1
max_depth                  6
max_leaves                20
min_child_weight         0.9
n_estimators             100
scale_pos_weight           5
subsample                0.8
tree_method             hist
test_f1             0.406173
train_f1             0.42688
Name: 70, dtype: object


In [110]:
params = {        
    'max_depth': 6, 
    'learning_rate': 0.1,
    'n_estimators': 100, 
    'gamma': 0, 
    'lambda': 0.1,
    'scale_pos_weight': 5,
    'subsample': 0.8,
    'max_leaves': 20,
    'min_child_weight': 0.9,
    'tree_method': 'hist'
}
model = xgb.XGBClassifier(**params)
model.fit(X_train_encoded, y_train)

print("Training results")
y_pred = model.predict(X_train_encoded)
print(confusion_matrix(y_train, y_pred))
print(classification_report(y_train, y_pred))

print("Validation results")
y_pred = model.predict(X_valid_encoded)
print(confusion_matrix(y_valid, y_pred))
print(classification_report(y_valid, y_pred))

Training results
[[26775 17746]
 [ 2635  7408]]
              precision    recall  f1-score   support

       False       0.91      0.60      0.72     44521
        True       0.29      0.74      0.42     10043

    accuracy                           0.63     54564
   macro avg       0.60      0.67      0.57     54564
weighted avg       0.80      0.63      0.67     54564

Validation results
[[3352 2226]
 [ 360  883]]
              precision    recall  f1-score   support

       False       0.90      0.60      0.72      5578
        True       0.28      0.71      0.41      1243

    accuracy                           0.62      6821
   macro avg       0.59      0.66      0.56      6821
weighted avg       0.79      0.62      0.66      6821



No se logro aumentar el F1-Score de validación pero si mantenerlo y reducir el F1-score de los datos de entrenamiento evitando asi que se sobre ajusten mucho a los datos de entrenamiento.

In [112]:
print("Test results")
y_pred = model.predict(X_test_encoded)
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))

Test results
[[3317 2176]
 [ 383  945]]
              precision    recall  f1-score   support

       False       0.90      0.60      0.72      5493
        True       0.30      0.71      0.42      1328

    accuracy                           0.62      6821
   macro avg       0.60      0.66      0.57      6821
weighted avg       0.78      0.62      0.66      6821



Podemos ver que en los datos de test el F1-score se generaliza más que en los de Validación por lo que ya nuestros parametros resultantes son:<br>
<br>
params = {       <br> 
    'max_depth': 6, <br>
    'learning_rate': 0.1,<br>
    'n_estimators': 100, <br>
    'gamma': 0, <br>
    'lambda': 0.1,<br>
    'scale_pos_weight': 5,<br>
    'subsample': 0.8,<br>
    'max_leaves': 20,<br>
    'min_child_weight': 0.9,<br>
    'tree_method': 'hist'<br>
}

# Conclusiones

En resumen, se ha logrado seleccionar el mejor modelo desarrollado por Juan en el notebook "to-expose" y se ha mejorado aún más su rendimiento al aplicar técnicas adicionales.<br> Además, se ha comprobado que el modelo supera al modelo base random.

Es importante destacar que, al agregar variables de importancia como predicciones del tiempo o días feriados, se podría obtener mejoras significativas en los resultados del modelo.

Por último, se ha logrado simplificar los métodos para que sean más fáciles de llevar a producción y se ha mejorado el tiempo de procesamiento de los datos.<br>
 En general, este trabajo ha sido fundamental para optimizar el modelo y hacerlo más útil y efectivo en situaciones de la vida real.

#